In [1]:
import pandas as pd

# = = =  = = =  = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

import time

# = = =  = = =  = = =

from bs4 import BeautifulSoup
import re

# = = =  = = =  = = =

import json

In [2]:
with open('./api.txt', 'r') as api_txt:
    api = api_txt.read()

api

'LXnMWcYQhipLAS7rImEzmZ3CkrU033FMha9cwVSngG4vbufTsAOCQQ=='

In [3]:
print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/2.product_url.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input['[Product] No'] = ['' if product_no == '' else int(product_no) for product_no in df_input['[Product] No']]

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：19705



,Url,[Product] No,[Product] Url
0,https://automechanika-shanghai.hk.messefrankfu...,1,https://automechanika-shanghai.hk.messefrankfu...
1,https://automechanika-shanghai.hk.messefrankfu...,2,https://automechanika-shanghai.hk.messefrankfu...
2,https://automechanika-shanghai.hk.messefrankfu...,3,https://automechanika-shanghai.hk.messefrankfu...
3,https://automechanika-shanghai.hk.messefrankfu...,4,https://automechanika-shanghai.hk.messefrankfu...
4,https://automechanika-shanghai.hk.messefrankfu...,5,https://automechanika-shanghai.hk.messefrankfu...
...,...,...,...
19700,https://automechanika-shanghai.hk.messefrankfu...,,
19701,https://automechanika-shanghai.hk.messefrankfu...,1,https://automechanika-shanghai.hk.messefrankfu...
19702,https://automechanika-shanghai.hk.messefrankfu...,2,https://automechanika-shanghai.hk.messefrankfu...
19703,https://automechanika-shanghai.hk.messefrankfu...,3,https://automechanika-shanghai.hk.messefrankfu...


In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 Url              https://automechanika-shanghai.hk.messefrankfu...
 [Product] No                                                     1
 [Product] Url    https://automechanika-shanghai.hk.messefrankfu...
 Name: 0, dtype: object)

In [5]:
url_request = '' if crawler_series['[Product] Url'] == '' else f'''https://api.messefrankfurt.com/service/esb_api/exhibitor-service/api/2.1/public/product/zh-CN/{crawler_series['[Product] Url'].removeprefix('https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/').removesuffix('.html')}/AUTOMECHANIKASHANGHAI/details'''

url_request

'https://api.messefrankfurt.com/service/esb_api/exhibitor-service/api/2.1/public/product/zh-CN/2e4fe27a-b72f-430e-90cc-20ee806ec900/mf_2_a3pk1a04eskp_4237825_10000100202401/AUTOMECHANIKASHANGHAI/details'

In [6]:
url_request == ''

False

In [7]:
crawler_retry = 0
while True:
    crawler_retry += 1
    print(crawler_retry)
    try:
        url_check = crawler_series['[Product] Url']

        resp = requests.get(url_request,
                            headers=get_header(**{'Apikey': api}),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200 or resp.status_code == 401 or resp.status_code == 400:
            dict_ = resp.json()

            if resp.status_code == 200:
                if dict_['success'] or dict_['systemErrors'] and dict_['systemErrors'][0]['statusCode'] == 400 and dict_['systemErrors'][0]['description'].strip() == 'FIND error: null':
                    break
            elif resp.status_code == 401 and dict_['status'] == 401 and dict_['message'].strip() == 'API GW: Unauthorized':
                print(f'fuck api {api}')
                time.sleep(3)
                with open('./api.txt', 'r') as api_txt:
                    api = api_txt.read()
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

dict_

1


{'success': True,
 'fieldErrors': [],
 'systemErrors': [],
 'message': None,
 'result': {'id': 'mf_2_a3pk1a04eskp_4237825_10000100202401',
  'name': '1',
  'description': {'images': ['product_pictures/10000100202401/A3PK1A04ESKP/1701160538978_110299494.jpeg'],
   'text': None},
  'specification': None,
  'shortDescription': None,
  'additionalImage': [],
  'exhibition': {'id': 'AUTOMECHANIKASHANGHAI',
   'name': 'Automechanika Shanghai 2024',
   'synonyme': None,
   'exhibitionHall': None,
   'href': 'automechanika-shanghai.hk.messefrankfurt.com',
   'logo': None,
   'startdate': '2024-12-02',
   'enddate': '2024-12-05'},
  'exhibitorRef': {'id': 'mf_2_a3pk1a04eskp',
   'fairId': 'AUTOMECHANIKASHANGHAI',
   'rewriteId': '2e4fe27a-b72f-430e-90cc-20ee806ec900',
   'name': None},
  'isTopProduct': None,
  'sortKey': '1',
  'productLink3d': None,
  'novelty': False,
  'pressRelease': None,
  'deeplink': '',
  'nextradeDeeplink': '',
  'contentdownloads': None,
  'videocontents': None}}

In [8]:
dict_['systemErrors'] and dict_['systemErrors'][0]['statusCode'] == 400 and dict_['systemErrors'][0]['description'].strip() == 'FIND error: null'

[]

In [9]:
dict_src, list_src = {}, dict_['result']['description']['images']
for i, src in enumerate(list_src):
    dict_src[str(i)] = f'https://exhibitorsearch.hk.messefrankfurt.com/images/large/{src.strip()}'

dict_src

{'0': 'https://exhibitorsearch.hk.messefrankfurt.com/images/large/product_pictures/10000100202401/A3PK1A04ESKP/1701160538978_110299494.jpeg'}

In [10]:
if dict_['result']['description']['text']:
    soup = BeautifulSoup(dict_['result']['description']['text'], 'lxml')
    description = re.sub(r'\t+', '\n', soup.get_text()).strip()
else:
    description = ''

print(description)

In [11]:
list_dict_correct.append({'Url': crawler_series['Url'],
                          '[Product] No': crawler_series['[Product] No'],
                          '[Product] Name': dict_['result']['name'].strip(),
                          '[Product] Description': description,
                          '[Product] Picture': '',
                          '[Product] Url': crawler_series['[Product] Url'],
                          '[Product] Json_Src': json.dumps(dict_src)})

list_dict_correct

[{'Url': 'https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/2e4fe27a-b72f-430e-90cc-20ee806ec900.html',
  '[Product] No': 1,
  '[Product] Name': '1',
  '[Product] Description': '',
  '[Product] Picture': '',
  '[Product] Url': 'https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/2e4fe27a-b72f-430e-90cc-20ee806ec900/mf_2_a3pk1a04eskp_4237825_10000100202401.html',
  '[Product] Json_Src': '{"0": "https://exhibitorsearch.hk.messefrankfurt.com/images/large/product_pictures/10000100202401/A3PK1A04ESKP/1701160538978_110299494.jpeg"}'}]

In [12]:
crawler_status = True

crawler_status

True

In [13]:
crawler_series['Check_Url'] = url_check

list_series_error.append(crawler_series)

list_series_error

[Url              https://automechanika-shanghai.hk.messefrankfu...
 [Product] No                                                     1
 [Product] Url    https://automechanika-shanghai.hk.messefrankfu...
 Check_Url        https://automechanika-shanghai.hk.messefrankfu...
 Name: 0, dtype: object]

In [14]:
print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Url', '[Product] No'],
                                                             ascending=[True, True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_product.xlsx', index=False)

df_correct if list_dict_correct else ''

Data Outputting...


,Url,[Product] No,[Product] Name,[Product] Description,[Product] Picture,[Product] Url,[Product] Json_Src
0,https://automechanika-shanghai.hk.messefrankfu...,1,1,,,https://automechanika-shanghai.hk.messefrankfu...,"{""0"": ""https://exhibitorsearch.hk.messefrankfu..."


In [15]:
print('Output Finished !')
print()

Output Finished !



In [16]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['Url', '[Product] No'],
                                                           ascending=[True, True],
                                                           ignore_index=True)
    df_error.to_excel('./test_product_error.xlsx', index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,Url,[Product] No,[Product] Url,Check_Url
0,https://automechanika-shanghai.hk.messefrankfu...,1,https://automechanika-shanghai.hk.messefrankfu...,https://automechanika-shanghai.hk.messefrankfu...
